# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Diego Bartolucci
- Nombre de alumno 2: Pilar Nilo

### **Link de repositorio de GitHub:** [Repositorio](https://github.com/DiegoBarto01/MDS7202-pili-barto)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [ ]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [ ]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

*Respuesta*

El ambiente es una partida de BlackJack donde el objetivo es vencer al 'dealer' obteniendo cartas que sumen lo más cercano a 21 sin pasarse. La idea es lograr alcanzar una mejor mano de cartas que las del 'dealer', cumpliendo la retricción de los 21 puntos.


Su formulción en MDP es la siguiente:



*   Estados: tenemos estado inicial y final

Inicial: se inicia el juego, el jugador suma sus cartas para que luego el 'Dealer' muestre las suyas y finalmente se puede utilizar un As.

Final: el final del juego sucede cuando el jugador pide una carta más y la suma resulta ser mayor a 21. Otro caso es cuando el jugador se queda con las cartas que tiene en mano.
*   Acciones: Hay dos acciones posibles

0: Pegar/Stick donde se queda con las cartas actuales

1: Golpear/Hit que implica robar una carta
*   Recompensas:

Las recompensas incluyen ganar 1 pto si se gana el juego, perder 1 pto si se pierde, 0 pts si se llega a un empate y 1.5 pts si se gana el juego con un natural blackjack.



#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

In [ ]:
#simulacion de escenario donde se escojen acciones aleatorias
def simul_escenario(env, episodios):
  recompensas=[]
  for episode in range(episodios):
    obs = env.reset()
    done = False
    while not done:
      action = env.action_space.sample()
      obs, reward, done,_, info = env.step(action)
      recompensas.append(reward)
  return recompensas




In [ ]:
#repeticion de la simulacion 5000 veces + promedio y desviasion de las recompensas
repeat_5000=simul_escenario(env,5000)
#saco promedio y desv estandar
promedio=np.mean(repeat_5000)
desviacion=np.std(repeat_5000)
print(f"El promedio de las recompensas es: {promedio}")
print(f"La desviacion estandar de las recompensas es: {desviacion}")

*Respuesta*

La performance de esta politica no es buena ya que el promedio de las recompensas es negativo indicando un escenario poco favorecedor ya que según los parametros de recompensas perdió la mayoría de las partidas jugadas. Además, la desviación estandar es alta lo que indica alta variación.

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [ ]:
#estoy entre box y multiProcessing
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
modelo=PPO("MlpPolicy",env,verbose=1)
modelo.learn(total_timesteps=int(2e5), progress_bar=True)
modelo.save("modelo_blackjack_1")

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [ ]:
#simulacion de 5000 veces más promedio y desviacion
def simul_escenario_2(env,modelo,episodios):
  recompensas=[]
  for episode in range(episodios):
    obs,_ = env.reset()
    episodio_recompensa=0
    done = False
    while not done:
      action,_state= modelo.predict(obs, deterministic=True)
      obs, reward, done,truncated,info= env.step(action)
      episodio_recompensa+=reward
      recompensas.append(episodio_recompensa)
  return recompensas

In [ ]:
repeat_5000_v2=simul_escenario_2(env,modelo,5000)
#saco promedio y desv estandar
promedio_2=np.mean(repeat_5000_v2)
desviacion_2=np.std(repeat_5000_v2)
print(f"El promedio de las recompensas es: {promedio_2}")
print(f"La desviacion estandar de las recompensas es: {desviacion_2}")

*Comparación de escenarios*

La performance del agente es mejor que el baseline ya que es más cercano a 0, pero sigue siendo un escenario desfavorable ya que indica que está perdiendo o empatando la mayoría de las jugadas.

Lo que si, baja la variabilidad del modelo.

#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [ ]:
#funcion que reciba un estado y retorne la accion del agente.
def get_return_action(model,state):
  action,_states= model.predict(state, deterministic=True)
  return action

In [ ]:
#escenario 1: suma de las cartas es 6, dealer muestra un 7, agente no tiene un as
escenario_1=np.array([6,7,0])
#escenario 2: suma de las cartas del agente es 19, dealer muestra un 3, agente tiene un as
escenario_2=np.array([19,3,1])

In [ ]:
#accion del escenario 1
accion_1=get_return_action(modelo,escenario_1)
#accion del escenario 2
accion_2=get_return_action(modelo,escenario_2)
print(f"La accion del escenario 1 es: {accion_1}")
print(f"La accion del escenario 2 es: {accion_2}")

*respuesta*

Si, son coherentes con las reglas del juego ya que en el primer escenario el dealer tiene mayor puntaje y pide otra carta para poder ganar y en el segundo escenario el dealer no supera la cantidad del agente por lo cual no se necesita de otra carta.

### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


*Se tuvo que utilizar LunarLander-v3 ya que arrojaba error la version2

In [ ]:
import gymnasium as gym
env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [ ]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

*respuesta*

El continuos=True indica que necesitamos tener un espacio de acción continuo en lugar de uno discreto

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

*respuesta*

El ambiente LunarLander es un problema de optimización de trayectoria de cohete.

Su formulacion en MDP se compone de:

*  Estados:
  * Inicial: la nave se encuentra en el centro superior de la ventana gráfica.
  * Final: el episodio termina el los siguientes casos--> a) aterrizaje y se estrella;
  b) aterrizaje sale de la ventana gráfica; c) modulo de aterrizaje no está despierto, i.e no se mueve y no choca con ningun otro cuerpo.
*  Acciones: Existen 4 tipos de acciones
    * 0: no hace nada
    * 1: se enciendo propulsor de orientación izquierda
    * 2: encender propulsor central
    * 3: encender propulsor de orientación derecha
*  Recompensas:
  * aumenta/disminuye cuanto más lento/rápido se mueve el modulo de aterrizaje de la plataforma de aterrizaje
  * aumenta/disminuye cuanto más lento/rápido se mueve el modulo de aterrizaje
  * disminuye cuanto más se inclina el modulo de aterrizaje (angulo no horizontal)
  * +10 puntos por cada pierna qu esté en contacto con el suelo
  * -0.03 puntos cada cuadro en que se activa un motor lateral
  * -0.3 puntos cada vez que se activa el motor principal
  * El episodio recibe +100/-100 por aterrizar de forma segura o estrellarse respectivamente.
  * Episodio se considera solución si obtiene al menos 200pts.


La diferencia con el ambiente black jack es que a partir de los puntos obtenidos en recompensas se considera si un episodio es optimo o no y con ello la solución, no así en el ambiente de black jack donde no requiere de un óptimo para definir una solución. Tambien las acciones disponibles para cada ambiente son distintas, en blackJack son 2 y en LunarLander 4.



#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [ ]:
#simulacion de escenario con acciones aleatorias
def simul_lunar_lander( env,episodios):
  recompensas=[]
  for episode in range(episodios):
    obs = env.reset()
    recompensa_episodios=0
    done = False
    while not done:
      action = env.action_space.sample()
      obs, reward, done,_, info = env.step(action)
      recompensa_episodios+=reward
      recompensas.append(recompensa_episodios)
  return recompensas

In [ ]:
#simulacion repetida 10 veces + promedio y desviacion estandar
repeat_10=simul_lunar_lander(env,10)
#saco promedio y desv estandar
promedio_3=np.mean(repeat_10)
desviacion_3=np.std(repeat_10)
print(f"El promedio de las recompensas es: {promedio_3}")
print(f"La desviacion estandar de las recompensas es: {desviacion_3}")

*respuesta*

La perfomance de la politica no es favorable ya que indica que en promedio se mueve lento el modulo de aterrizaje, quizás se estrelló o simplemente una mala ejecución dentro del episodio ya que el promedio es negativo.

La desviación estandar indica que tiene alta variabilidad

#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [ ]:
modelo_lunar=PPO('MlpPolicy',env,verbose=0)
modelo_lunar.learn(total_timesteps=int(1e4),progress_bar=True)
modelo_lunar.save("modelo_lunar_1")

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [ ]:
def simul_lunar_lander_2(env,modelo,episodios):
  recompensas=[]
  for episode in range(episodios):
    obs,_ = env.reset()
    episodio_recompensa=0
    done = False
    while not done:
      action,_state= modelo.predict(obs, deterministic=True)
      obs, reward, done,truncated,info= env.step(action)
      episodio_recompensa+=reward
    recompensas.append(episodio_recompensa)
  return recompensas

In [ ]:
recompensa_lunar=simul_lunar_lander_2(env,modelo_lunar,10)
#saco promedio y desv estandar
promedio_4=np.mean(recompensa_lunar)
desviacion_4=np.std(recompensa_lunar)
print(f"El promedio de las recompensas es: {promedio_4}")
print(f"La desviacion estandar de las recompensas es: {desviacion_4}")

la perfomance mejora en cuanto al promedio ya que indica que se cometen menos errores en la obtencion de las recompensas.

En cambio la desviacion estandar baja.

#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

In [ ]:
modelo_modificado=PPO('MlpPolicy',env,verbose=0,learning_rate=0.0001,batch_size=32,seed=400)
modelo_modificado.learn(total_timesteps=int(1e6),progress_bar=True)
modelo_modificado.save("modelo_lunar_modificado")

In [ ]:
recompensa_modelo_modificado=simul_lunar_lander_2(env,modelo_modificado,10)
#saco promedio y desv estandar
promedio_5=np.mean(recompensa_modelo_modificado)
desviacion_5=np.std(recompensa_modelo_modificado)
print(f"El promedio de las recompensas es: {promedio_5}")
print(f"La desviacion estandar de las recompensas es: {desviacion_5}")

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

In [ ]:
# Se carga el modelo visto en clases
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # modelo de lenguaje
    temperature=0, # probabilidad de "respuestas creativas"
    max_tokens=None, # sin tope de tokens
    timeout=None, # sin timeout
    max_retries=2, # número máximo de intentos
)

llm

### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [ ]:
%pip install --upgrade --quiet PyPDF2

In [ ]:
import PyPDF2

doc_paths = ['53701409.pdf','Super_Mario_Bros.pdf','Mario_interior_OK_MK.0.pdf', 'Mario_fuera_del_reino_de_Nintendo.pdf','Descripcion_Mario_Bros.pdf'] # rellenar con los path a sus documentos

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

In [ ]:
%pip install --upgrade --quiet faiss-cpu langchain_community pypdf

In [ ]:
#Se guardan los documentos en una lista
from langchain_community.document_loaders import PyPDFLoader
docus = []
for doc in doc_paths:
  loader = PyPDFLoader(doc)
  docus += loader.load()
  print(f"Se cargó el documento {doc}")

In [ ]:
len(docus)

In [ ]:
docus[0]

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docus)
splits[:5]

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento
vectorstore

#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", # método de búsqueda
                                     search_kwargs={"k": 3}, # n° documentos a recuperar
                                     )
retriever

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever_chain = retriever | format_docs

#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

In [ ]:
from langchain_core.prompts import PromptTemplate

# noten como ahora existe el parámetro de context!
rag_template = '''
Eres un asistente experto en los videojuegos, específicamente en el videojuego llamado "Super Mario Bros" y su historia.
Tu único rol es contestar preguntas del usuario a partir de información relevante que te sea proporcionada.
Responde siempre de la forma más completa posible y usando toda la información entregada.
Responde sólo lo que te pregunten a partir de la información relevante, NUNCA inventes una respuesta.

Información relevante: {context}
Pregunta: {question}
Respuesta útil:
'''

rag_prompt = PromptTemplate.from_template(rag_template)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)

In [ ]:
#Pregunta y respuesta 1
question1 = "Quién en Mario?"
res_esp1 = "Mario es el protagonista de la saga de juegos Super Mario Bros e icono de la compañía de videojuegos Japonesa Nintendo."
response1 = rag_chain.invoke(question1)
print("Respuesta esperada: " + str(res_esp1))
print("Respuesta obtenida: " + str(response1))

In [ ]:
#Pregunta y respuesta 2
question2 = "Qué relación tiene Mario con Luigi?"
res_esp2 = "Luigi es el hermano menor de Mario"
response2 = rag_chain.invoke(question2)
print("Respuesta esperada: " + str(res_esp2))
print("Respuesta obtenida: " + str(response2))

In [ ]:
#Pregunta y respuesta 3
question3 = "Quién es Bowser?"
res_esp3 = "Bowser es el enemigo de Mario"
response3= rag_chain.invoke(question3)
print("Respuesta esperada: " + str(res_esp3))
print("Respuesta obtenida: " + str(response3))

Observando las respuestas se puede concluir que responde correctamente la pregunta realizada e incluso desarrolla más aún la idea con los datos proporcionados utilizando la mayor cantidad de información que posee.


#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

In [ ]:
#Empezamos generando listas con las preguntas a hacer y respuestas esperadas
questions_list = [question1, question2, question3]
expected_answers = [res_esp1, res_esp2, res_esp3]

#Posteriormente creamos una función para evaluar el funcionamiento cambiando parámetros
#Se crea esta función para no tener que repetir el código una y otra vez
def sensibilidad(chunk_size: int = 500, cant_chunks: int = 3, search_type: str = 'similarity'):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=50)
  splits = text_splitter.split_documents(docus)

  embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
  vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento

  retriever = vectorstore.as_retriever(search_type=search_type, # método de búsqueda
                                     search_kwargs={"k": cant_chunks}, # n° documentos a recuperar
                                     )
  retriever_chain = retriever | format_docs

  rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
  )

  for i in range(len(questions_list)):
    response = rag_chain.invoke(questions_list[i])
    print("Pregunta: " + str(questions_list[i]))
    print("Respuesta esperada: " + str(expected_answers[i]))
    print("Respuesta obtenida: " + str(response))

**SENSIBILIDAD ANTE EL TAMAÑO DE LOS CHUNKS**

In [ ]:
#Ahora empezamos a variar los parámetros para evaluar la sensibilidad del modelo a estos
sensibilidad(chunk_size=250)

In [ ]:
sensibilidad(chunk_size=1000)

Se observa que al achicar el tamaño de los chunks las respuestas son más precisas pero sujetas a mucho detalle repercutiendo en respuestas que nos entregan información que no pedimos. Al aumentar el tamaño de los chunks se ven respuestas más concisas y precisas.

**SENSIBILIDAD ANTE LA CANTIDAD DE CHUNKS**

In [ ]:
sensibilidad(cant_chunks=1)

In [ ]:
sensibilidad(cant_chunks=5)

Se observa que al disminuir la cantidad de chunks la información recopilada de los documentos es menor, llegando a entregar respuestas que, a pesar de estar la información, están incompletas o bien incorrectas. Al aumentar la cantidad de chunks se observa como las respuestas entregadas son más completas ya que se abarca más información presente en los documentos.

**SENSIBILIDAD ANTE EL TIPO DE BÚSQUEDA**

In [ ]:
sensibilidad(search_type='mmr')

Se observan respuestas similares al uso de similarity, pero se puede observar una mayor diversidad en el contenido de la respuesta que con este otro método. Esto debido a que usando mmr se utilizan chunks distintos entre sí generando una respuesta más variada.

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool_tavily = TavilySearchResults(max_results = 1) # inicializamos tool
tools = []
tools.append(tool_tavily) # guardamos las tools en una lista

#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

In [ ]:
%pip install wikipedia

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
wiki_api_wrapper = WikipediaAPIWrapper(lang='es', top_k_results=1, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
tools.append(wiki_tool)

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

In [ ]:
from langchain import hub

react_prompt = hub.pull("hwchase17/react") # template de ReAct
print(react_prompt.template)

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor

agent = create_react_agent(llm, tools, react_prompt) # primero inicializamos el agente ReAct
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # lo transformamos a AgentExecutor para habilitar la ejecución de tools
agent_executor

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

In [ ]:
response = agent_executor.invoke({"input": "Quién es Taylor Swift?"}) #Hacer una pregunta
print(response["output"])

In [ ]:
response = agent_executor.invoke({"input": "Quién dijo en una película la frase: Santa cachucha ya dijo la frase?"}) #Hacer otra pregunta
print(response["output"])

El agente debería utilizar wikipedia en casos que la información se pueda encontrar ahí como casos de historia, ciencia, etc. Mientras que tavily debería ser utilizado para casos más específicos pudiendo encontrar esta información en la web.

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

In [ ]:
from langchain.tools import tool

@tool
def RAG(input:str) -> str:
  """Usa el modelo RAG para responder el input"""
  return rag_chain.invoke(input)

@tool
def agent(input:str) -> str:
  """Usa el modelo ReAct para responder el input"""
  return agent_executor.invoke({"input": input})["output"]

#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

In [ ]:
supervisor_agent_prompt = PromptTemplate.from_template(
    """
    Eres un supervisor de preguntas.
    Tu único rol es seleccionar que acción tomar para responder correctamente la pregunta:
    - 'mario': Cuando la pregunta esté relacionada con Super Mario Bros, sus personajes o los videojuegos.
    - 'internet': Cuando la pregunta es de conocimiento general y puede ser respondida utilizando wikipedia o usando tavily.
    - 'otro': Todo aquella pregunta que no esté contenida en las categorías anteriores.

    No respondas con más de una palabra y no incluyas.


    {question}


    Categoría:"""
)

supervisor_agent_chain = (
    supervisor_agent_prompt
    | llm
    | StrOutputParser()
)

supervisor_agent_chain.invoke({"question": "Quién fue Mozart??"})

In [ ]:
redirect_agent_prompt = PromptTemplate.from_template(
    """
    Eres un asistente de redireccionamiento de preguntas de usuarios.
    Vas a recibir una pregunta del usuario, tu único rol es indicar cuando no puedes responder su pregunta y redireccionar al usuario
    para que te pregunte sobre videojuegos, sobre tu videojuego favorito Super Mario Bros o bien información de cultura general que
    pueda ser encontrada en wikipedia o usando tavily.

    Recuerda ser amable y cordial en tu respuesta.

    Pregunta: {question}
    Respuesta cordial:"""
)

redirect_agent_chain = (
    redirect_agent_prompt
    | llm
    | StrOutputParser()
)

redirect_agent_chain.invoke({"question": "Quién fue Mozart?"})

In [ ]:
def route_question(question):
  '''
  Recibe una pregunta de usuario.
  Rutea la pregunta al agente respectivo y responde de manera acorde.
  '''

  topic = supervisor_agent_chain.invoke({"question": question}) # enrutamiento

  if "mario" in topic: # si la pregunta es de mario o videojuegos, utilizar rag
      return rag_chain.invoke(question)
  elif "internet" in topic: # si la pregunta es de cultura general, utilizar agente
      return agent_executor.invoke({"input": question})["output"]
  else: # de lo contrario, redireccionar pregunta
      return redirect_agent_chain.invoke({"question": question})

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

In [ ]:
route_question("Quién en Mario?")

In [ ]:
route_question("Qué relación tiene Mario con Luigi?")

In [ ]:
route_question("Quién es Taylor Swift")

In [ ]:
route_question("Quién dijo en una película la frase: Santa cachucha ya dijo la frase?")

Las respuestas entregadas no varían mucho, siendo la mayor diferencia que no constesta la última pregunta al ser muy rebuscada a pesar de que se encuentra en internet usando Tavily considera que corresponde a la categoría de "otro", principalmente porque no considera que esta respuesta sea conocimiento general como le fue pedido.


#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

El agente supervisor utiliza un modelo de lenguaje para razonar de manera dinámica y decidir qué herramientas emplear, lo que le permite adaptarse con facilidad a diferentes situaciones y resolver problemas en tiempo real. En contraste, el Router opera siguiendo un conjunto de reglas establecidas, lo que lo hace más rápido y eficiente, pero menos capaz de enfrentar escenarios ambiguos o no planificados. Aunque el agente supervisor destaca por su capacidad para abordar casos complejos sin depender de reglas fijas, este enfoque requiere un mayor esfuerzo computacional debido a su uso constante del modelo de lenguaje.

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [ ]:
%pip install --upgrade --quiet gradio

Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [ ]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = ... # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202 - PiliBarto", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil >:)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )